# Symbolic Dalitz-Plot Decomposition

```{autolink-concat}
```

In [ ]:
from __future__ import annotations

import itertools
from typing import Iterable

import qrules
import sympy as sp
from ampform.kinematics.phasespace import Kallen
from ampform.sympy import (
    UnevaluatedExpression,
    create_expression,
    implement_doit_method,
    make_commutative,
)
from IPython.display import Latex, Markdown

from ampform_dpd import DalitzPlotDecompositionBuilder, simplify_latex_rendering
from ampform_dpd.decay import (
    IsobarNode,
    Particle,
    ThreeBodyDecay,
    ThreeBodyDecayChain,
)
from ampform_dpd.io import as_markdown_table, aslatex
from ampform_dpd.spin import filter_parity_violating_ls, generate_ls_couplings

simplify_latex_rendering()

## Decay definition

In [ ]:
PDG = qrules.load_pdg()
PARTICLE_DB = {
    p.name: Particle(
        name=p.name,
        latex=p.latex,
        spin=p.spin,
        parity=int(p.parity),
        mass=p.mass,
        width=p.width,
    )
    for p in PDG
    if p.parity is not None
}
PARTICLE_DB["Lambda(2000)"] = Particle(
    name="Lambda(2000)",
    latex=R"\Lambda(2000)",
    spin=0.5,
    parity=-1,
    mass=2.0,
    width=(0.020 - 0.400) / 2,
)
Λc = PARTICLE_DB["Lambda(c)+"]
p = PARTICLE_DB["p"]
π = PARTICLE_DB["pi+"]
K = PARTICLE_DB["K-"]
PARTICLE_TO_ID = {Λc: 0, p: 1, π: 2, K: 3}
Markdown(as_markdown_table(list(PARTICLE_TO_ID)))

In [ ]:
resonance_names = [
    "Lambda(1405)",
    "Lambda(1520)",
    "Lambda(1600)",
    "Lambda(1670)",
    "Lambda(1690)",
    "Lambda(2000)",
    "Delta(1232)+",
    "Delta(1600)+",
    "Delta(1700)+",
    "K(0)*(700)+",
    "K*(892)0",
    "K(2)*(1430)0",
]
resonances = [PARTICLE_DB[name] for name in resonance_names]
Markdown(as_markdown_table(resonances))

In [ ]:
def load_three_body_decay(
    resonance_names: Iterable[str],
    particle_definitions: dict[str, Particle],
    min_ls: bool = True,
) -> ThreeBodyDecay:
    resonances = [particle_definitions[name] for name in resonance_names]
    chains: list[ThreeBodyDecayChain] = []
    for res in resonances:
        chains.extend(_create_isobar(res, min_ls))
    return ThreeBodyDecay(
        states={state_id: particle for particle, state_id in PARTICLE_TO_ID.items()},
        chains=tuple(chains),
    )


def _create_isobar(resonance: Particle, min_ls: bool) -> list[ThreeBodyDecayChain]:
    if resonance.name.startswith("K"):
        child1, child2, spectator = π, K, p
    elif resonance.name.startswith("L"):
        child1, child2, spectator = K, p, π
    elif resonance.name.startswith("D"):
        child1, child2, spectator = p, π, K
    else:
        raise NotImplementedError
    prod_ls_couplings = _generate_ls(Λc, resonance, spectator, conserve_parity=False)
    dec_ls_couplings = _generate_ls(resonance, child1, child2, conserve_parity=True)
    if min_ls:
        decay = IsobarNode(
            parent=Λc,
            child1=IsobarNode(
                parent=resonance,
                child1=child1,
                child2=child2,
                interaction=min(dec_ls_couplings),
            ),
            child2=spectator,
            interaction=min(prod_ls_couplings),
        )
        return [ThreeBodyDecayChain(decay)]
    chains = []
    for dec_ls, prod_ls in itertools.product(dec_ls_couplings, prod_ls_couplings):
        decay = IsobarNode(
            parent=Λc,
            child1=IsobarNode(
                parent=resonance,
                child1=child1,
                child2=child2,
                interaction=dec_ls,
            ),
            child2=spectator,
            interaction=prod_ls,
        )
        chains.append(ThreeBodyDecayChain(decay))
    return chains


def _generate_ls(
    parent: Particle, child1: Particle, child2: Particle, conserve_parity: bool
) -> list[tuple[int, sp.Rational]]:
    ls = generate_ls_couplings(parent.spin, child1.spin, child2.spin)
    if conserve_parity:
        return filter_parity_violating_ls(
            ls, parent.parity, child1.parity, child2.parity
        )
    return ls


DECAY = load_three_body_decay(
    resonance_names,
    particle_definitions=PARTICLE_DB,
    min_ls=True,
)
Latex(aslatex(DECAY, with_jp=True))

## Lineshapes for dynamics

In [ ]:
@make_commutative
@implement_doit_method
class BreitWignerMinL(UnevaluatedExpression):
    def __new__(
        cls,
        s,
        decaying_mass,
        spectator_mass,
        resonance_mass,
        resonance_width,
        child2_mass,
        child1_mass,
        l_dec,
        l_prod,
        R_dec,
        R_prod,
    ):
        return create_expression(
            cls,
            s,
            decaying_mass,
            spectator_mass,
            resonance_mass,
            resonance_width,
            child2_mass,
            child1_mass,
            l_dec,
            l_prod,
            R_dec,
            R_prod,
        )

    def evaluate(self):
        s, m_top, m_spec, m0, Γ0, m1, m2, l_dec, l_prod, R_dec, R_prod = self.args
        q = Q(s, m_top, m_spec)
        q0 = Q(m0**2, m_top, m_spec)
        p = P(s, m1, m2)
        p0 = P(m0**2, m1, m2)
        width = EnergyDependentWidth(s, m0, Γ0, m1, m2, l_dec, R_dec)
        return sp.Mul(
            (q / q0) ** l_prod,
            BlattWeisskopf(q * R_prod, l_prod) / BlattWeisskopf(q0 * R_prod, l_prod),
            1 / (m0**2 - s - sp.I * m0 * width),
            (p / p0) ** l_dec,
            BlattWeisskopf(p * R_dec, l_dec) / BlattWeisskopf(p0 * R_dec, l_dec),
            evaluate=False,
        )

    def _latex(self, printer, *args) -> str:
        s = printer._print(self.args[0])
        return Rf"\mathcal{{R}}\left({s}\right)"


@make_commutative
@implement_doit_method
class P(UnevaluatedExpression):
    def __new__(cls, s, mi, mj, **hints):
        return create_expression(cls, s, mi, mj, **hints)

    def evaluate(self):
        s, mi, mj = self.args
        return sp.sqrt(Kallen(s, mi**2, mj**2)) / (2 * sp.sqrt(s))

    def _latex(self, printer, *args):
        s, mi, mj = map(printer._print, self.args)
        return Rf"p_{{{mi},{mj}}}\left({s}\right)"


@make_commutative
@implement_doit_method
class Q(UnevaluatedExpression):
    def __new__(cls, s, m0, mk, **hints):
        return create_expression(cls, s, m0, mk, **hints)

    def evaluate(self):
        s, m0, mk = self.args
        return sp.sqrt(Kallen(s, m0**2, mk**2)) / (2 * m0)  # <-- not s!

    def _latex(self, printer, *args):
        s, m0, mk = map(printer._print, self.args)
        return Rf"q_{{{m0},{mk}}}\left({s}\right)"


@make_commutative
@implement_doit_method
class EnergyDependentWidth(UnevaluatedExpression):
    def __new__(cls, s, m0, Γ0, m1, m2, L, R):
        return create_expression(cls, s, m0, Γ0, m1, m2, L, R)

    def evaluate(self):
        s, m0, Γ0, m1, m2, L, R = self.args
        p = P(s, m1, m2)
        p0 = P(m0**2, m1, m2)
        ff = BlattWeisskopf(p * R, L) ** 2
        ff0 = BlattWeisskopf(p0 * R, L) ** 2
        return sp.Mul(
            Γ0,
            (p / p0) ** (2 * L + 1),
            m0 / sp.sqrt(s),
            ff / ff0,
            evaluate=False,
        )

    def _latex(self, printer, *args) -> str:
        s = printer._print(self.args[0])
        return Rf"\Gamma\left({s}\right)"


@make_commutative
@implement_doit_method
class BlattWeisskopf(UnevaluatedExpression):
    def __new__(cls, z, L, **hints):
        return create_expression(cls, z, L, **hints)

    def evaluate(self) -> sp.Piecewise:
        z, L = self.args
        cases = {
            0: 1,
            1: 1 / (1 + z**2),
            2: 1 / (9 + 3 * z**2 + z**4),
        }
        return sp.Piecewise(
            *[(sp.sqrt(expr), sp.Eq(L, l_val)) for l_val, expr in cases.items()]
        )

    def _latex(self, printer, *args):
        z, L = map(printer._print, self.args)
        return Rf"F_{{{L}}}\left({z}\right)"


s, m0, Γ0, m1, m2 = sp.symbols("s m0 Gamma0 m1 m2", nonnegative=True)
m_top, m_spec = sp.symbols(R"m_\mathrm{top} m_\mathrm{spectator}")
R_dec, R_prod = sp.symbols(R"R_\mathrm{res} R_{\Lambda_c}")
l_Λc, l_R = sp.symbols(R"l_{\Lambda_c} l_R", integer=True, positive=True)
bw = BreitWignerMinL(s, m_top, m_spec, m0, Γ0, m1, m2, l_R, l_Λc, R_dec, R_prod)
Latex(aslatex({bw: bw.doit(deep=False)}))

In [ ]:
def formulate_breit_wigner(
    decay_chain: ThreeBodyDecayChain,
) -> tuple[BreitWignerMinL, dict[sp.Symbol, float]]:
    s = _get_mandelstam_s(decay_chain)
    child1_mass, child2_mass = map(_to_mass_symbol, decay_chain.decay_products)
    l_dec = sp.Rational(decay_chain.outgoing_ls.L)
    l_prod = sp.Rational(decay_chain.incoming_ls.L)
    parent_mass = sp.Symbol(f"m_{{{decay_chain.parent.name}}}")
    spectator_mass = sp.Symbol(f"m_{{{decay_chain.spectator.name}}}")
    resonance_mass = sp.Symbol(f"m_{{{decay_chain.resonance.name}}}")
    resonance_width = sp.Symbol(Rf"\Gamma_{{{decay_chain.resonance.name}}}")
    R_dec = sp.Symbol(R"R_\mathrm{res}")
    R_prod = sp.Symbol(R"R_{\Lambda_c}")
    parameter_defaults = {
        parent_mass: decay_chain.parent.mass,
        spectator_mass: decay_chain.spectator.mass,
        resonance_mass: decay_chain.resonance.mass,
        resonance_width: decay_chain.resonance.width,
        child1_mass: decay_chain.decay_products[0].mass,
        child2_mass: decay_chain.decay_products[1].mass,
        # https://github.com/ComPWA/polarimetry/pull/11#issuecomment-1128784376
        R_dec: 1.5,
        R_prod: 5,
    }
    dynamics = BreitWignerMinL(
        s,
        parent_mass,
        spectator_mass,
        resonance_mass,
        resonance_width,
        child1_mass,
        child2_mass,
        l_dec,
        l_prod,
        R_dec,
        R_prod,
    )
    return dynamics, parameter_defaults


def _get_mandelstam_s(decay: ThreeBodyDecayChain) -> sp.Symbol:
    s1, s2, s3 = sp.symbols("sigma1:4", nonnegative=True)
    m1, m2, m3 = map(_to_mass_symbol, [p, π, K])
    decay_masses = {_to_mass_symbol(p) for p in decay.decay_products}
    if decay_masses == {m2, m3}:
        return s1
    if decay_masses == {m1, m3}:
        return s2
    if decay_masses == {m1, m2}:
        return s3
    raise NotImplementedError(
        f"Cannot find Mandelstam variable for {''.join(decay_masses)}"
    )


def _to_mass_symbol(particle: Particle) -> sp.Symbol:
    state_id = PARTICLE_TO_ID.get(particle)
    if state_id is not None:
        return sp.Symbol(f"m{state_id}", nonnegative=True)
    return sp.Symbol(f"m_{{{particle.name}}}", nonnegative=True)

## Model formulation

In [ ]:
model_builder = DalitzPlotDecompositionBuilder(DECAY, min_ls=True)
for chain in model_builder.decay.chains:
    model_builder.dynamics_choices.register_builder(chain, formulate_breit_wigner)
model = model_builder.formulate(reference_subsystem=1)
model.intensity

In [ ]:
Latex(aslatex(model.amplitudes))

```{toctree}
---
hidden:
maxdepth: 2
---
API <api/ampform_dpd>
```